In [1]:
pwd

'/Users/vmuccion/switchdrive/NLP_transformers/ClimateEducation/cgpt_classification'

In [3]:
import pandas as pd
import os
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, EarlyStoppingCallback, Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoModelForPreTraining
from collections import OrderedDict
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [4]:
os.environ["WANDB_DISABLED"] = "true"

In [11]:
path = r"C:\Saeid\Prj100\SA_45_DSS_embrace"
train_file = "seen1.xlsx"
test_file = "unseen.xlsx"
train_path = os.path.join(path, train_file)
test_path = os.path.join(path, test_file)
cgpt_path = r"C:\Saeid\Prj100\SA_45_DSS_embrace\models\climateGPT2_ready_RL\pytorch_model.bin" # the path to bin file of cgpt((this is ckeygpt do we want cgpt or this?))
output_path = r"C:\Saeid\Prj100\SA_45_DSS_embrace\relevancy_cgpt_save"

In [12]:
set_epoch = 100

train_data = pd.read_excel(train_path)
print(train_data.head())

test_data = pd.read_excel(test_path)
print(test_data.head())

train_data.describe()

test_data.describe()

ready_to_train = train_data[["Abstract", "Topic"]]
print(ready_to_train.head())

train, test = train_test_split(train_data[["Abstract", "Relevant"]], test_size=0.33, random_state=42)

print(train.head())
print(test.head())
class relevancyDataset(Dataset):

    def __init__(self, data, tokenizer):
        x = data["Abstract"].tolist()
        label = data["Relevant"].tolist()

        self.tokenizer = tokenizer
        self.x = x
        self.label = label

    # ---------------------------------------------#

    def __len__(self):
        return len(self.label)

    # ---------------------------------------------#

    def __getitem__(self, i):
        encodings_dict = tokenizer(self.x[i],
                                   truncation=True,
                                   max_length=1500,
                                   padding="max_length")

        input_ids = encodings_dict['input_ids']
        attention_mask = encodings_dict['attention_mask']

        return {'label':torch.tensor(self.label[i]),
            'input_ids': torch.tensor(input_ids),
            'attention_mask': torch.tensor(attention_mask)}


                           DOI  \
0   10.1016/j.nedt.2021.105144   
1   10.1007/s11422-021-10080-6   
2     10.1016/j.pt.2010.06.009   
3            10.1111/jnu.12326   
4  10.1016/j.jsurg.2020.11.001   

                                               Title  \
0  A mixed method study on global warming, climat...   
1  Global challenges need attention now: educatin...   
2  Soil-transmitted helminthiases: implications o...   
3  Climate Change, Climate Justice, and Environme...   
4  The Role of Preoperative Briefing and Postoper...   

                                            Abstract  \
0  AIM: This study aims to evaluate the knowledge...   
1  This Editorial sets the stage for 18 papers on...   
2  Soil-transmitted helminthiases (STHs) collecti...   
3  PURPOSE: Climate change is an emerging challen...   
4  OBJECTIVE: To study the impact of a new preope...   

                            Source title  PubYear  \
0                  Nurse Education Today     2021   
1  Cultural Stu

In [13]:
base_model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token

tt = GPT2Tokenizer.from_pretrained(base_model)
tt.pad_token = tt.eos_token

train_data_set = relevancyDataset(train, tokenizer)
test_data_set = relevancyDataset(test, tokenizer)


model_final = AutoModelForSequenceClassification.from_pretrained(output_path)
model_final.config.pad_token_id = model_final.config.eos_token_id
xd = []
dx = []
for k, i in enumerate(test_data["Abstract"]):
    print(str(k)+"/"+str(len(test_data["Abstract"])))
    inputs = tokenizer(i,truncation=True,max_length=750,padding="max_length", return_tensors='pt')
    logits = model_final(**inputs).logits

    softmax= torch.nn.Softmax()

    predicted_class_id = logits.argmax().item()
    prcnt = round(softmax(logits).max().item()*100, 2)


    xd.append(predicted_class_id)
    dx.append(prcnt)
req = pd.DataFrame({"Abstract":test_data["Abstract"], "Relevant": xd, "DOI":test_data["DOI"], "Title":test_data["Title"], "Source title":test_data["Source title"],"PubYear":test_data["PubYear"],"Authors":test_data["Authors"],"Times cited":test_data["Times cited"],"Probability":dx})
print(req.head())

0/14794


C:\Users\saeed\AppData\Local\Temp/ipykernel_26908/578589286.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prcnt = round(softmax(logits).max().item()*100, 2)


1/14794
2/14794
3/14794
4/14794
5/14794
6/14794
7/14794
8/14794
9/14794
10/14794
11/14794
12/14794
13/14794
14/14794
15/14794
16/14794
17/14794
18/14794
19/14794
20/14794
21/14794
22/14794
23/14794
24/14794
25/14794
26/14794
27/14794
28/14794
29/14794
30/14794
31/14794
32/14794
33/14794
34/14794
35/14794
36/14794
37/14794
38/14794
39/14794
40/14794
41/14794
42/14794
43/14794
44/14794
45/14794
46/14794
47/14794
48/14794
49/14794
50/14794
51/14794
52/14794
53/14794
54/14794
55/14794
56/14794
57/14794
58/14794
59/14794
60/14794
61/14794
62/14794
63/14794
64/14794
65/14794
66/14794
67/14794
68/14794
69/14794
70/14794
71/14794
72/14794
73/14794
74/14794
75/14794
76/14794
77/14794
78/14794
79/14794
80/14794
81/14794
82/14794
83/14794
84/14794
85/14794
86/14794
87/14794
88/14794
89/14794
90/14794
91/14794
92/14794
93/14794
94/14794
95/14794
96/14794
97/14794
98/14794
99/14794
100/14794
101/14794
102/14794
103/14794
104/14794
105/14794
106/14794
107/14794
108/14794
109/14794
110/14794
111/1479

KeyboardInterrupt: 

In [ ]:
ff = []
for k, i in enumerate(train_data["Abstract"]):
    print(str(k)+"/"+str(len(train_data["Abstract"])))
    ff.append(100.0)
)
qer = pd.DataFrame({"Abstract":train_data["Abstract"], "Relevant": train_data["Relevant"], "DOI":train_data["DOI"], "Title":train_data["Title"], "Source title":train_data["Source title"],"PubYear":train_data["PubYear"],"Authors":train_data["Authors"],"Times cited":train_data["Times cited"],"Probability":ff})

print("END!!")
print(qer.head())

result = pd.concat([qer, req])
print(result.head())
rls = shuffle(result)
print(rls.head())
rls.to_excel(r"C:\Saeid\Prj100\SA_45_DSS_embrace\relevancy_save\output_relevant_unseen_seen_merge_shuffle.xlsx")